In [89]:
import pandas as pd
import numpy as np
from IPython.display import HTML
from pandas_summary import DataFrameSummary
import tensorflow
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Concatenate,Input, Dense, Embedding, merge, LSTM, concatenate, Flatten, Dropout, Lambda, BatchNormalization
from keras.models import Model, load_model
from keras.regularizers import l2
from keras.optimizers import SGD, RMSprop, Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error


In [90]:
dataAll = pd.read_csv('/Users/ruben.arana/git/beat-bookies/data/dataWithFeatures2006.csv')

In [78]:
bettorsData = pd.read_csv('/Users/ruben.arana/git/beat-bookies/data/combinedData.csv', low_memory=False)

In [92]:
bettorsData.head()

,GAME_ID,SEASON,DAY_OF_WEEK,MONTH,DAY,YEAR,HOUR,VISITOR_TEAM,VISITOR_POINTS,HOME_TEAM,...,HOME_STL%,HOME_BLK%,HOME_TOV%,HOME_USG%,HOME_ORtg,HOME_DRtg,MIN_LINE,MAX_LINE,AVERAGE_LINE,PREDICTIONS
0,200611010BOS,2007,Wed,11,1,2006,7:30 pm,New Orleans,91,Boston,...,5.0,8.2,17.1,100.0,86.4,90.3,195.5,198.5,197.500000,778.200928
1,200611010CHA,2007,Wed,11,1,2006,7:00 pm,Indiana,106,Charlotte,...,7.9,10.0,19.3,100.0,97.8,104.7,192.5,193.5,193.433333,772.372253
2,200611010CLE,2007,Wed,11,1,2006,8:00 pm,Washington,94,Cleveland,...,4.5,6.3,14.9,100.0,108.5,105.1,194.5,194.5,194.500000,777.187988
3,200611010DET,2007,Wed,11,1,2006,8:00 pm,Milwaukee,105,Detroit,...,3.1,11.4,20.2,100.0,101.3,109.6,189.5,194.5,194.166667,748.821167
4,200611010GSW,2007,Wed,11,1,2006,7:30 pm,L.A. Lakers,110,Golden State,...,6.1,5.8,9.1,100.0,99.8,112.1,199.5,205.5,204.566667,709.507446


In [93]:
dataAll.head()

,VISITOR_TEAM,HOME_TEAM,VISITOR_AS_VISITOR_FG,VISITOR_AS_VISITOR_FGA,VISITOR_AS_VISITOR_FG%,VISITOR_AS_VISITOR_3P,VISITOR_AS_VISITOR_3PA,VISITOR_AS_VISITOR_3P%,VISITOR_AS_VISITOR_FT,VISITOR_AS_VISITOR_FTA,...,LAST_3_HOME_AS_VISITOR_AGAINST_DRB%,LAST_3_HOME_AS_VISITOR_AGAINST_TRB%,LAST_3_HOME_AS_VISITOR_AGAINST_AST%,LAST_3_HOME_AS_VISITOR_AGAINST_STL%,LAST_3_HOME_AS_VISITOR_AGAINST_BLK%,LAST_3_HOME_AS_VISITOR_AGAINST_TOV%,LAST_3_HOME_AS_VISITOR_AGAINST_USG%,LAST_3_HOME_AS_VISITOR_AGAINST_ORtg,LAST_3_HOME_AS_VISITOR_AGAINST_DRtg,TOTAL_POINTS
0,1179,1192,35.555556,81.111111,0.438556,3.111111,12.777778,0.247333,17.888889,24.666667,...,71.800000,47.700000,55.633333,6.700000,4.066667,11.600000,100.0,108.366667,111.366667,178
1,1191,1146,34.500000,80.500000,0.429200,7.600000,18.900000,0.402400,19.700000,26.000000,...,76.666667,52.766667,59.400000,6.933333,10.200000,14.266667,100.0,108.766667,103.466667,205
2,1169,1197,31.285714,78.857143,0.396571,3.571429,15.000000,0.236429,21.285714,27.714286,...,70.433333,49.500000,56.733333,6.033333,6.566667,10.666667,100.0,114.600000,103.233333,191
3,1186,1183,37.875000,83.500000,0.453125,4.625000,17.500000,0.270625,17.125000,21.750000,...,76.666667,52.200000,50.500000,5.166667,5.833333,11.500000,100.0,110.100000,116.400000,202
4,1144,1156,35.666667,76.666667,0.465667,6.333333,19.666667,0.320333,25.000000,33.666667,...,71.566667,53.500000,63.766667,10.033333,6.200000,13.233333,100.0,116.600000,112.833333,208


In [101]:
model = load_model('/Users/ruben.arana/git/beat-bookies/models/2006_2_001_BATCH.h5')

In [102]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 770)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 770)               3080      
_________________________________________________________________
dense_19 (Dense)             (None, 370)               285270    
_________________________________________________________________
dense_20 (Dense)             (None, 150)               55650     
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 151       
Total params: 344,151
Trainable params: 342,611
Non-trainable params: 1,540
_________________________________________________________________


In [103]:
labels = ['TOTAL_POINTS']

input_data = dataAll.loc[:,dataAll.columns.difference(labels)]

In [104]:
input_data.head()

,HOME_AS_HOME_3P,HOME_AS_HOME_3P%,HOME_AS_HOME_3PA,HOME_AS_HOME_3PAr,HOME_AS_HOME_AGAINST_3P,HOME_AS_HOME_AGAINST_3P%,HOME_AS_HOME_AGAINST_3PA,HOME_AS_HOME_AGAINST_3PAr,HOME_AS_HOME_AGAINST_AST,HOME_AS_HOME_AGAINST_AST%,...,VISITOR_AS_VISITOR_STL,VISITOR_AS_VISITOR_STL%,VISITOR_AS_VISITOR_TOV,VISITOR_AS_VISITOR_TOV%,VISITOR_AS_VISITOR_TRB,VISITOR_AS_VISITOR_TRB%,VISITOR_AS_VISITOR_TS%,VISITOR_AS_VISITOR_USG%,VISITOR_AS_VISITOR_eFG%,VISITOR_TEAM
0,6.888889,0.427556,16.333333,0.208000,3.333333,0.279111,11.111111,0.142444,21.777778,61.144444,...,5.666667,6.300000,15.000000,14.022222,44.000000,52.677778,0.500111,100.0,0.457778,1179
1,3.833333,0.310833,12.500000,0.162500,4.333333,0.273500,16.666667,0.212500,19.166667,56.466667,...,7.800000,8.120000,17.200000,15.810000,41.400000,47.470000,0.525500,100.0,0.476700,1191
2,6.333333,0.362556,17.888889,0.227111,5.555556,0.378333,14.444444,0.186778,18.888889,54.033333,...,7.857143,8.642857,14.571429,13.785714,41.285714,46.442857,0.480714,100.0,0.418714,1169
3,6.428571,0.370571,17.428571,0.219571,4.142857,0.309429,13.571429,0.175857,19.285714,55.457143,...,7.750000,8.150000,15.250000,14.150000,36.500000,44.787500,0.523750,100.0,0.481250,1186
4,8.272727,0.387364,21.636364,0.255909,6.636364,0.308909,21.090909,0.250727,23.181818,63.563636,...,6.000000,6.500000,15.000000,14.166667,36.666667,45.266667,0.563000,100.0,0.508000,1144


In [105]:
# I've got my predictions now
predictions = model.predict(input_data)

bettorsData['PREDICTIONS'] = predictions
predictions

array([[ 173.74008179],
       [ 214.93605042],
       [ 186.60980225],
       ..., 
       [ 186.07398987],
       [ 197.72892761],
       [ 190.65455627]], dtype=float32)

In [85]:
bettorsData.head()

,GAME_ID,SEASON,DAY_OF_WEEK,MONTH,DAY,YEAR,HOUR,VISITOR_TEAM,VISITOR_POINTS,HOME_TEAM,...,HOME_STL%,HOME_BLK%,HOME_TOV%,HOME_USG%,HOME_ORtg,HOME_DRtg,MIN_LINE,MAX_LINE,AVERAGE_LINE,PREDICTIONS
0,200611010BOS,2007,Wed,11,1,2006,7:30 pm,New Orleans,91,Boston,...,5.0,8.2,17.1,100.0,86.4,90.3,195.5,198.5,197.500000,778.200928
1,200611010CHA,2007,Wed,11,1,2006,7:00 pm,Indiana,106,Charlotte,...,7.9,10.0,19.3,100.0,97.8,104.7,192.5,193.5,193.433333,772.372253
2,200611010CLE,2007,Wed,11,1,2006,8:00 pm,Washington,94,Cleveland,...,4.5,6.3,14.9,100.0,108.5,105.1,194.5,194.5,194.500000,777.187988
3,200611010DET,2007,Wed,11,1,2006,8:00 pm,Milwaukee,105,Detroit,...,3.1,11.4,20.2,100.0,101.3,109.6,189.5,194.5,194.166667,748.821167
4,200611010GSW,2007,Wed,11,1,2006,7:30 pm,L.A. Lakers,110,Golden State,...,6.1,5.8,9.1,100.0,99.8,112.1,199.5,205.5,204.566667,709.507446


In [114]:

sum = 0;
games = 0;
for index, row in bettorsData.iterrows():
    totalPoints = row['HOME_POINTS'] + row['VISITOR_POINTS']
    prediction = row['PREDICTIONS']
    averageLine = row['AVERAGE_LINE']
    if (((prediction > averageLine) & (totalPoints > averageLine)) | ((prediction < averageLine) & (totalPoints < averageLine))):
        sum = sum + 1.9
        games = games +1
    else :
        sum = sum - 1
        games = games +1


print ('WINNINGS: ' + str(sum) )
print ('GAMES: ' + str(games) )

WINNINGS: 1638.1000000000276
GAMES: 1201


In [117]:
sum = 0;
games = 0;
for index, row in bettorsData.iterrows():
    totalPoints = row['HOME_POINTS'] + row['VISITOR_POINTS']
    prediction = row['PREDICTIONS']
    averageLine = row['AVERAGE_LINE']
    if (((prediction > averageLine + 5) & (totalPoints > averageLine)) | ((prediction < averageLine - 5) & (totalPoints < averageLine))):
        sum = sum + 1.9
        games = games +1
    elif (((prediction > averageLine) & (totalPoints > averageLine)) | ((prediction < averageLine) & (totalPoints < averageLine))):
        # nothing
        games = games
    else :
        sum = sum - 1
        games = games +1


print ('WINNINGS: ' + str(sum) )
print ('GAMES: ' + str(games) )

WINNINGS: 1280.9000000000096
GAMES: 1013


In [106]:
mean_squared_error(bettorsData["AVERAGE_LINE"], bettorsData["VISITOR_POINTS"] + bettorsData["HOME_POINTS"])

326.50530849978981

In [107]:
mean_squared_error(bettorsData["PREDICTIONS"], bettorsData["VISITOR_POINTS"] + bettorsData["HOME_POINTS"])

140.8804463203318